In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import networkx as nx

In [4]:
from pygsp import graphs, filters, plotting, utils

In [5]:
# Create a random directed graph
G = nx.gnp_random_graph(100, 0.5, directed=True)
A = nx.to_numpy_array(G)

In [12]:
g = graphs.DiGraph(A, q=0.01)

> /home/akumar/nse/pygsp/pygsp/graphs/graph.py(1083)compute_laplacian()
   1081 
   1082         pdb.set_trace()
-> 1083         self.L = D - np.multiply(Q, Wsymm)
   1084 
   1085     def dirichlet_energy(self, x):



In [ ]:
# Verify the dirchilet energy of the eigenvectors. Then test localization of wavelet bases. 